# Vehicle Speed (target) Aggregation: 'Moving Average' Strategy

* save directory: `../datasets/per-vehicle-moving-average/<aggregation type>/window-<some window choice>-<datestring>`

In [1]:
import shap
import pandas as pd
import numpy as np

import time

import seaborn as sns

import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder

from glob import glob
import os
import joblib
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

import multiprocessing as mp
from datetime import datetime
import itertools

In [2]:
import psutil
    
ram_gb = psutil.virtual_memory().total / 2**30 # total physical memory in bytes
print(f"RAM: {ram_gb:.2f} GB")

RAM: 251.54 GB


## Read from source

In [3]:
# get paths
root = '../datasets/processed'
# source_date = '2022-05-30'
# source_date = '2022-05-24'
source_date = '2023-06-15'
glob_path = os.path.join(root, source_date, '*.csv')
filepaths = glob(glob_path)

# check
filepaths[0]

'../datasets/processed/2023-06-15/123200872603.csv'

In [4]:
def dedup_based_on_trace_data(df, verbose=False):
    landuse_cols = [
        col for col in df.columns if col.startswith("pix_")
    ]
    trace_columns = [i for i in df.columns if i not in landuse_cols]
    init_count = df.shape[0]
    df = df.drop_duplicates(subset=trace_columns)
    new_count = df.shape[0]
    
    if verbose:
        print(f"Before: {init_count}, After: {new_count}")
        
    return df

In [5]:
def get_periods(row, ref_index, thresh):
    """Return period counts for thresh"""
    timedeltas = (row.name - ref_index).total_seconds()
    # >= 0 includes self
    periods = np.sum((timedeltas <= thresh) & (timedeltas >= 0))
    return periods

def get_agg_speed(row, group, thresh, agg='mean'):
    """Return a aggregated speed value according to thresh and agg."""
    timedeltas = (row.name - group.index).total_seconds()
    mask = (timedeltas <= thresh) & (timedeltas >= 0)
    # >= 0 includes self
    if agg=='mean':
        return group[mask]['instant_speed'].mean()
    elif agg=='median':
        return group[mask]['instant_speed'].median()    
    else:
        raise "Enter valid agg."

## Parallelize

In [6]:
def mp_get_periods(chunk, ref_index, thresh):
    chunk['num_periods'] = chunk.apply(
        lambda row: get_periods(row, ref_index, thresh=thresh), axis=1)
    return chunk

def mp_agg_speed(chunk, full_df, thresh, agg):
    chunk['agg_speed'] = chunk.apply(
        lambda row: get_agg_speed(row, full_df, thresh=thresh, agg=agg), axis=1)
    return chunk

# parallelize agg function
def parallelize_get_periods(df, func, thresh):
    """Parallelize mp_elevation and mp_building_counts functions"""
    ref_index = df.index
    cpus = mp.cpu_count()
    df_chunks = np.array_split(df, cpus)
    pool = mp.Pool(processes=cpus)
    chunk_processes = [pool.apply_async(func, args=(chunk, ref_index, thresh)) for chunk in df_chunks]
    df_results = []
    for chunk in chunk_processes:
        res = chunk.get()
        df_results.append(res)
    df_out = pd.concat(df_results)
    return df_out

def parallelize_agg_speed(df, func, thresh, agg):
    cpus = mp.cpu_count()
    df_chunks = np.array_split(df, cpus)
    pool = mp.Pool(processes=cpus)
    chunk_processes = [pool.apply_async(func, args=(chunk, df, thresh, agg)) for chunk in df_chunks]
    df_results = []
    for chunk in chunk_processes:
        res = chunk.get()
        df_results.append(res)
    df_out = pd.concat(df_results)
    return df_out


In [8]:
## use this for progress_apply method:
# tqdm.pandas()
# date = '2022-05-27' # alternatively use now date for new runs
# recompute speed through "moving average" method, get median/mean speed in window
# saves a new csv with columns for number of periods (num_periods) and agg vehicle speed (agg_speed)

date = str(datetime.date(datetime.now())) # now date
print(date)

# aggs = ['median', 'mean']
# lookback_windows = [30, 60, 120] # seconds, can be changed

aggs = ['mean']
lookback_windows = [60]

iters = list(itertools.product(aggs, lookback_windows))

for iter_ in iters:
    agg = iter_[0]
    lookback_window = iter_[1]
    
    for index, path in tqdm(enumerate(filepaths)):
        name = path.split('/')[-1].split('.')[0]
        group = pd.read_csv(path)

        group = dedup_based_on_trace_data(group, verbose=True)

        # prep save paths
        save_dir = f'../datasets/per-vehicle-moving-average/{agg}-window-{lookback_window}-{date}'
        
        os.makedirs(save_dir, exist_ok=True)
        save_path = os.path.join(save_dir, f"{name}.csv")
        
        if not os.path.exists(save_path):
            
            # get datetime index
            group['datetime'] = pd.to_datetime(group['datetime'])
            group = group.sort_values(by='datetime')
            group = group.set_index('datetime')

            group = parallelize_get_periods(
                df=group, 
                func=mp_get_periods, 
                thresh=lookback_window)

            group = parallelize_agg_speed(
                df=group, 
                func=mp_agg_speed, 
                thresh=lookback_window, 
                agg=agg) 

            group = group.reset_index()

            # save
            group.to_csv(save_path, index=False)

2023-06-17


0it [00:00, ?it/s]

Before: 53175, After: 53175


1it [00:34, 34.16s/it]

Before: 153493, After: 153492


...

198it [13:51:04, 331.34s/it]

Before: 275829, After: 275829


199it [13:58:01, 357.16s/it]

Before: 132807, After: 132807


200it [13:59:40, 251.90s/it]
